In [9]:
import requests
import json
import pandas as pd
from os import makedirs
from os.path import join, exists
from datetime import date, timedelta

/Users/ashish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [7]:
API_ENDPOINT = 'https://newsapi.org/v2/everything'
with open('newsapi-org.txt', 'r') as f:
    API_KEY = f.read().strip()
    
DATA_DIR = join('data', 'ny_abc_articles')
makedirs(DATA_DIR, exist_ok=True)

In [8]:
API_KEY

'3d2142f3c3b44c2596d5912c319e4f0f'

In [35]:
search_params = {
    'from': '2017-09-01',
    'to': '2018-03-30',
    'sortBy': 'popularity',
    'pageSize': 100,
    'apiKey': API_KEY,
    'sources' : 'abc-news',
    'domains' : 'nytimes.com'
}

In [36]:
filename = join(DATA_DIR, 'ny_abc.json')
    
#if file not exists then only download it
pages_data = []
if not exists(filename):
    response = requests.get(API_ENDPOINT, search_params)
    data = response.json()
    results = data['articles']
    pages_data.extend(results)
        
    #write all data to json file
    with open(filename, 'w') as f:
        print("Writing to {} ...".format(filename))
        f.write(json.dumps(pages_data, indent=2))    

Writing to data/ny_abc_articles/ny_abc.json ...


In [7]:
#extract all valid sources from newsapi
#Save the json once and then read it later to do filtering of columns and 
API_ENDPOINT = 'https://newsapi.org/v2/sources'
DATA_DIR = 'data'
search_params = {
    'apiKey': API_KEY
}
with open('newsapi-org.txt', 'r') as f:
    API_KEY = f.read().strip()
    
src_file = join(DATA_DIR, 'newsapi_credible_src.json')
src_data = []
if not exists(src_file):
    response = requests.get(API_ENDPOINT, search_params)
    data = response.json()
    srclist = data['sources']
    src_data.extend(srclist)
    
#write all data to json file
with open(src_file, 'w') as f:
    print("Writing to {} ...".format(src_file))
    f.write(json.dumps(src_data, indent=2))    


Writing to data/newsapi_credible_src.json ...


In [11]:
df_src = pd.read_json(src_file)

In [13]:
df_src.shape

(138, 7)

In [12]:
df_src.head()

,category,country,description,id,language,name,url
0,general,us,"Your trusted source for breaking news, analysi...",abc-news,en,ABC News,http://abcnews.go.com
1,general,au,"Australia's most trusted source of local, nati...",abc-news-au,en,ABC News (AU),http://www.abc.net.au/news
2,general,no,Norges ledende nettavis med alltid oppdaterte ...,aftenposten,no,Aftenposten,https://www.aftenposten.no
3,general,us,"News, analysis from the Middle East and worldw...",al-jazeera-english,en,Al Jazeera English,http://www.aljazeera.com
4,general,it,"Agenzia ANSA: ultime notizie, foto, video e ap...",ansa,it,ANSA.it,http://www.ansa.it


In [14]:
#mask to select only english language sources afor now 
mask = df_src['language'] == 'en'
df_src = df_src[mask]

In [15]:
df_src.shape

(91, 7)

In [16]:
df_src = df_src.drop(['category', 'country', 'description', 'language'], axis=1)

In [17]:
df_src.head()

,id,name,url
0,abc-news,ABC News,http://abcnews.go.com
1,abc-news-au,ABC News (AU),http://www.abc.net.au/news
3,al-jazeera-english,Al Jazeera English,http://www.aljazeera.com
6,ars-technica,Ars Technica,http://arstechnica.com
8,associated-press,Associated Press,https://apnews.com/


In [18]:
df_src.to_csv('data/credible.csv', sep=',')

In [ ]:

# # An example of sample response 
# https://newsapi.org/v2/sources?apiKey=3d2142f3c3b44c2596d5912c319e4f0f
# {
# "status": "ok",
# -"sources": [
# -{
# "id": "abc-news",
# "name": "ABC News",
# "description": "Your trusted source for breaking news, analysis, exclusive interviews, headlines, and videos at ABCNews.com.",
# "url": "http://abcnews.go.com",
# "category": "general",
# "language": "en",
# "country": "us"
# },
# -{
# "id": "abc-news-au",
# "name": "ABC News (AU)",
# "description": "Australia's most trusted source of local, national and world news. Comprehensive, independent, in-depth analysis, the latest business, sport, weather and more.",
# "url": "http://www.abc.net.au/news",
# "category": "general",
# "language": "en",
# "country": "au"
# }
# ]
# }